# Caso 2 - CA417
## Jimena Chacón - B81999

In [30]:
#pip install arch

     -------------------------------------- 925.2/925.2 kB 2.0 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [25]:
import numpy as np
import pandas as pd
import scipy.stats as stats
from arch import arch_model
import yfinance as yf

###  Supuestos

Particularmente en el Metodo EWMA se tomará el ponderador $\lambda$ igual a 0.94, esto dado a que este número captura bien las fluctuaciones diarias sin reaccionar drásticamente a picos. Es un equilibrio entre la volatilidad histórica y la reciente.


In [138]:
archivo = "C:/Users/jimec/OneDrive/Documents/UCR/Riesgos en Entidades Financieras/MSFT.csv"
datos_msft = pd.read_csv(archivo)

arch2 = "C:/Users/jimec/OneDrive/Documents/UCR/Riesgos en Entidades Financieras/NVDA.csv"
datos_nvda = pd.read_csv(arch2)

#datos_nvda = yf.download('NVDA', start='2023-09-05', end='2024-09-05')['Adj Close']

### Inciso 1)

In [92]:
rendimientos_msft = np.log(datos_msft['Adj Close'] / datos_msft['Adj Close'].shift(1)).dropna()

acciones_msft = 1000

precio_msft = datos_msft['Adj Close'].iloc[-1] # Últimos precios

valor_portafolio_msft = acciones_msft * precio_msft

print("Valor del portafolio de Microsoft:",valor_portafolio_msft)

Valor del portafolio de Microsoft: 408390.015


#### Estimador Insesgado

In [99]:
media_msft = rendimientos_msft.mean() #Esperanza de los rendimientos
var_msft = rendimientos_msft.var() #Varianza de los rendimientos
media_perdidas_msft = media_msft*-valor_portafolio_msft

#Estimador insesgado 
var_perdidas_msft = var_msft* (valor_portafolio_msft)**2
print("Varianza con estimador insesgado:", var_msft)

nivel_confianza = 0.95
Z_alpha = stats.norm.ppf(nivel_confianza)

# Cálculo del VaR paramétrico
VaR = media_perdidas_msft + np.sqrt(var_perdidas_msft) * Z_alpha
print("VaR con estimador insesgado: ",VaR)

#Cálculo de ES paramétrico
phi_Z_alpha = stats.norm.pdf(Z_alpha)

# Calcular el ES usando la fórmula
#ES = media_perdidas_msft + np.sqrt(var_perdidas_msft) * (phi_Z_alpha / (1 - nivel_confianza))
ES= media_perdidas_msft + (np.sqrt(var_perdidas_msft)/(np.sqrt(2*np.pi)*(1-nivel_confianza)))*np.exp(-((Z_alpha)**2)/2)
print("ES con estimador insesgado: ",ES)


Varianza con estimador insesgado: 0.0001581446968560946
VaR con estimador insesgado:  8102.726372198612
ES con estimador insesgado:  10248.73952786968


#### Método EWMA

In [103]:
rendi_array = np.array(rendimientos_msft)
var_msft_ewma=0
suma=0
for i in range(1,len(rendi_array)):
    suma=((pond)**(i-1)*(rendi_array[-i]-media_msft)**2)+suma
var_msft_ewma = suma*((1-pond)/(1-(pond)**len(rendi_array))) #Varianza rendimientos
print("Varianza con EWMA:", var_msft_ewma)

#EWMA 
var_perdidas_msft = var_msft_ewma* (valor_portafolio_msft)**2

# Cálculo del VaR paramétrico
VaR_ewma = media_perdidas_msft + np.sqrt(var_perdidas_msft) * Z_alpha
print("VaR con método EWMA: ",VaR_ewma)

# Calcular el ES usando la fórmula
ES_ewma= media_perdidas_msft + (np.sqrt(var_perdidas_msft)/(np.sqrt(2*np.pi)*(1-nivel_confianza)))*np.exp(-((Z_alpha)**2)/2)
print("ES con método EWMA: ",ES_ewma)

Varianza con EWMA: 0.00013533215648067634
VaR con método EWMA:  7469.727613256005
ES con método EWMA:  9454.933546709424


#### GARCH

In [104]:
garch_mod = arch_model(rendimientos_msft, vol='Garch',p=1,q=1,rescale=False).fit(disp="off")
var_msft_garch = garch_mod.conditional_volatility.iloc[-1]**2
print("Varianza con GARCH:", var_msft_garch)

var_perdidas_msft = var_msft_garch* (valor_portafolio_msft)**2

# Cálculo del VaR paramétrico
VaR_ewma = media_perdidas_msft + np.sqrt(var_perdidas_msft) * Z_alpha
print("VaR con método GARCH: ",VaR_ewma)

# Calcular el ES usando la fórmula
ES_ewma= media_perdidas_msft + (np.sqrt(var_perdidas_msft)/(np.sqrt(2*np.pi)*(1-nivel_confianza)))*np.exp(-((Z_alpha)**2)/2)
print("ES con método GARCH: ",ES_ewma)

Varianza con GARCH: 0.00015437817845337287
VaR con método GARCH:  8001.523124926977
ES con método GARCH:  10121.826572888125


### Inciso 2)

In [140]:
rendimientos_nvda =  np.log(datos_nvda['Adj Close'] / datos_nvda['Adj Close'].shift(1)).dropna()

acciones_nvda = 2550

precio_nvda = datos_nvda['Adj Close'].iloc[-1] # Últimos precios

valor_portafolio_nvda = acciones_nvda * precio_nvda
print("Valor del portafolio de NVidia:",valor_portafolio_nvda)

valor_portafolio_combinado = acciones_msft * precio_msft + acciones_nvda * precio_nvda

print("Valor del portafolio combinado de Microsoft y NVidia:",valor_portafolio_combinado)

Valor del portafolio de NVidia: 273385.49744999997
Valor del portafolio combinado de Microsoft y NVidia: 681775.51245


#### Estimador Insesgado

In [145]:
media_nvda = rendimientos_nvda.mean() #Esperanza de los rendimientos
var_nvda = rendimientos_nvda.var() #Varianza de los rendimientos

correlacion = rendimientos_nvda.corr(rendimientos_msft)
media_perdidas_combinado = media_msft*-valor_portafolio_msft - media_nvda * valor_portafolio_nvda
var_perdidas_combinado = (valor_portafolio_msft**2)*var_msft + (valor_portafolio_nvda**2)*var_nvda + 2 *correlacion* valor_portafolio_msft*valor_portafolio_nvda*np.sqrt(var_msft)*np.sqrt(var_nvda)

# Cálculo del VaR paramétrico
VaR = media_perdidas_combinado + np.sqrt(var_perdidas_combinado) * Z_alpha
print("VaR combinado con estimador insesgado: ",VaR)

# Calcular el ES usando la fórmula
ES= media_perdidas_combinado + (np.sqrt(var_perdidas_combinado)/(np.sqrt(2*np.pi)*(1-nivel_confianza)))*np.exp(-((Z_alpha)**2)/2)
print("ES combinado con estimador insesgado: ",ES)

0.03193271526320969
VaR combinado con estimador insesgado:  28732.432243756513
ES combinado con estimador insesgado:  36583.217056756956


In [147]:
valor_portafolio_nvda

273385.49744999997

#### Método EWMA

In [150]:
rendi_array2 = np.array(rendimientos_nvda)
var_nvda_ewma=0
suma=0
for i in range(1,len(rendi_array2)):
    suma=((pond)**(i-1)*(rendi_array2[-i]-media_nvda)**2)+suma
var_nvda_ewma = suma*((1-pond)/(1-(pond)**len(rendi_array2))) #Varianza rendimientos


var_perdidas_combinado = (valor_portafolio_msft**2)*var_msft_ewma + (valor_portafolio_nvda**2)*var_nvda_ewma + 2 *correlacion* valor_portafolio_msft*valor_portafolio_nvda*np.sqrt(var_msft_ewma)*np.sqrt(var_nvda_ewma)


# Cálculo del VaR paramétrico
VaR_combinado = media_perdidas_combinado + np.sqrt(var_perdidas_combinado) * Z_alpha
print("VaR combinado con método EWMA: ",VaR_combinado)

# Calcular el ES usando la fórmula
ES_combinado= media_perdidas_combinado + (np.sqrt(var_perdidas_combinado)/(np.sqrt(2*np.pi)*(1-nivel_confianza)))*np.exp(-((Z_alpha)**2)/2)
print("ES combinado con método EWMA: ",ES_combinado)

VaR combinado con método EWMA:  22392.679043944725
ES combinado con método EWMA:  28632.910775750854


#### GARCH

In [149]:
garch_mod = arch_model(rendimientos_nvda, vol='Garch',p=1,q=1,rescale=False).fit(disp="off")
var_nvda_garch = garch_mod.conditional_volatility.iloc[-1]**2

var_perdidas_nvda = var_nvda_garch* (valor_portafolio_nvda)**2

var_perdidas_combinado = (valor_portafolio_msft**2)*var_msft_garch + (valor_portafolio_nvda**2)*var_nvda_garch + 2 *correlacion* valor_portafolio_msft*valor_portafolio_nvda*np.sqrt(var_msft_garch)*np.sqrt(var_nvda_garch)


# Cálculo del VaR paramétrico
VaR_garch = media_perdidas_combinado + np.sqrt(var_perdidas_combinado) * Z_alpha
print("VaR con método GARCH: ",VaR_garch)

# Calcular el ES usando la fórmula
ES_garch= media_perdidas_combinado + (np.sqrt(var_perdidas_combinado)/(np.sqrt(2*np.pi)*(1-nivel_confianza)))*np.exp(-((Z_alpha)**2)/2)
print("ES con método GARCH: ",ES_garch)

Varianza con GARCH: 0.0015891872141400186
VaR con método GARCH:  20849.470964310764
ES con método GARCH:  26697.665585326486
